In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from IPython.core.display import HTML
from bokeh.io import output_notebook, show

output_notebook()
pd.options.display.float_format = '{:,.5f}'.format
display(HTML("<style>img[src*='#left'] { float: left; }</style>"))
HTML('<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">')

Loading BokehJS ...

Backtesting of a UNI v2 LUNC/USTC LP strategy
---

In this notebook we pull data from Binance and we run a static LP strategy.  
The pools are arbed at the beginning of each simulation step if it is profitable to do so.  
At the end of the simulation we compute and display various metrics for the strategy.    

![backtesting](https://letianzj.github.io/quanttrading-backtest/backtest_structure.png#left)  

Connection to Binance
---


The binance client allows to connect to Binance in order to pull historical data.  
In this example, credentials are saved as environment variables.

In [2]:
from terra_algo_backtest.binance_loader import new_binance_client

# Replace these with your Binance API key and secret
client = new_binance_client(os.getenv("BINANCE_API_KEY"), os.getenv("BINANCE_API_SECRET"))

Creating trades
---

We use OHLVC data and create a buy order for a green candle or a sell order for a red one.  
If the pair is not listed, we create a composite one using data from each token (component) against a pivot, for example BUSD.  
In the example below we use data from LUNC/BUSD and USTC/BUSD to create trades for LUNC/USTC every hour.  
We then display, some metrics for our pair such as the price, the average price for the period or measures of how cointegrated the component prices are.

In [3]:
from binance import Client
from terra_algo_backtest.utils import format_df
from terra_algo_backtest.plotting import new_trade_figure

base_pair, quote_pair = 'LUNC/BUSD', 'USTC/BUSD'
start, end = '2023-03-01 00:00:00', '2023-06-28 23:59:59'
frequency = Client.KLINE_INTERVAL_1HOUR

df_lunc = client.get_trade_data(base_pair, frequency, start, end)
df_ustc = client.get_trade_data(quote_pair, frequency, start, end)
df_trades = client.create_trade_data(base_pair, quote_pair, 0.001, frequency, start, end)

You can also load trades from csv files:

In [4]:
# load sample from csv file
df_lunc = pd.read_csv("lunc_busd_trades.csv", parse_dates = ["trade_date"], index_col="trade_date")
df_ustc = pd.read_csv("ustc_busd_trades.csv", parse_dates = ["trade_date"], index_col="trade_date")
df_trades = pd.read_csv("lunc_ustc_trades.csv", parse_dates = ["trade_date"], index_col="trade_date")

In [5]:
show(new_trade_figure(df_lunc, df_ustc, df_trades, "LUNC/USTC"))

Function 'plot_price_ratio' executed in 0.0365s
Function 'plot_scatterplot' executed in 0.0146s
Function 'new_trade_figure' executed in 0.0861s


Backtesting
---

We create a pair with an amount of liquidity in USD. We then run the backtesting between 01/03/2023 and 28/06/2023.  
Various metrics such as the P&L, price impact or impermanent loss are plotted.  
For the impermanent loss, the solid line shows the actual one from the simulation eg. the loss incured by people trading against the DEX. 
Whereas the dash line shows the theorical using this [formula](https://medium.com/auditless/how-to-calculate-impermanent-loss-full-derivation-803e8b2497b7).    

⚠️ **All units are expressed in quote currency unless explicitly specified. So in this example that's USTC**  

![backtesting](https://mtr-cdn.com/images/backtesting_strategies_on_forex.width-648.jpg#left)  

In [6]:
from terra_algo_backtest.market import MarketQuote, new_market
from terra_algo_backtest.simulation import swap_simulation
from terra_algo_backtest.plotting import new_simulation_figure
from terra_algo_backtest.exec_engine import ConstantProductEngine
from terra_algo_backtest.strategy import SimpleUniV2Strategy

liquidity_usd = 1000000
# LUNC/BUSD market price
base = MarketQuote(base_pair, df_trades.price_1.iloc[0])
# USTC/BUSD market price
quote = MarketQuote(quote_pair, df_trades.price_2.iloc[0])
# create a 1,000,000 USD market for LUNC/USTC with 1% swap fee 
mkt = new_market(liquidity_usd, quote, base, 0.003)
# create a cp swap execution engine
cp_amm = ConstantProductEngine(mkt)
# uni v2 lp strategy with arb to keep the price of the pool in line with mkt
strategy = SimpleUniV2Strategy(cp_amm, arb_enabled=True)
# run simulation
simul = swap_simulation(df_trades, strategy)
# display results
show(new_simulation_figure(mkt, simul))

Function 'trade_summary' executed in 0.0094s
Function 'sim_results' executed in 0.0393s
Function 'swap_simulation' executed in 0.2343s


In [7]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# display breakdown results
simul["breakdown"].head(20)

,side,price,price_impact,qty_received,mid_price,mkt_price,spread,avg_price,current_base,current_quote,cp_invariant,total_fees_paid_quote,total_volume_base,total_volume_quote,asset_base_pct,hold_portfolio,current_portfolio,trade_pnl,total_pnl,roi,impermanent_loss,mkt_price_ratio,arb_profit,volume_base,volume_quote,fees_paid_quote,trade_pnl_pct,fees_pnl_pct,total_arb_profit
trade_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-03-01 03:00:00,sell,0.00607,0.00000,0.00000,0.00607,0.00607,-0.00000,0.00609,"3,032,876,379.95878","18,422,957.95328","55,874,554,025,485,456.00000",0.00000,0.00002,-0.00000,0.50000,"36,845,915.90657","36,845,915.90657",0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00002,-0.00000,0.00000,0.00000,0.00000,0.00000
2023-03-01 03:00:00,sell,0.00607,0.00009,"1,738.65245",0.00607,0.00607,0.00000,0.00609,"3,033,162,632.34333","18,421,219.30083","55,874,554,025,485,456.00000",5.23165,"286,252.38457","-1,743.88410",0.50000,"36,842,443.99740","36,842,438.60166",-5.39574,-0.16408,-0.00000,0.00000,1.00000,0.00000,"286,252.38455","-1,743.88410",5.23165,-0.00000,0.00000,0.00000
2023-03-01 07:00:00,sell,0.00607,0.00053,"9,693.20792",0.00607,0.00607,-0.00000,0.00609,"3,034,759,516.59340","18,411,526.09292","55,874,554,025,485,464.00000",34.39878,"1,883,136.63464","-11,466.25914",0.50000,"36,823,093.67834","36,823,052.18583",-41.49250,-7.09373,-0.00000,-0.00000,0.99876,845.77149,"1,596,884.25007","-9,722.37504",29.16713,-0.00000,0.00000,845.77150
2023-03-01 07:00:00,sell,0.00607,0.00001,216.08289,0.00607,0.00607,0.00000,0.00609,"3,034,795,133.80979","18,411,310.01002","55,874,554,025,485,456.00000",35.04898,"1,918,753.85103","-11,682.99224",0.50000,"36,822,662.43346","36,822,620.02005",-42.41341,-7.36443,-0.00000,-0.00000,0.99876,0.00000,"35,617.21639",-216.73309,0.65020,-0.00000,0.00000,845.77150
2023-03-01 09:00:00,sell,0.00603,0.00547,"100,693.79481",0.00600,0.00600,-0.00000,0.00606,"3,051,484,088.17889","18,310,616.21522","55,874,554,025,485,448.00000",338.03933,"18,607,708.22014","-112,679.77740",0.50000,"36,622,255.52079","36,621,232.43043","-1,023.09035",-685.05102,-0.00002,-0.00002,0.98781,"92,327.42886","16,688,954.36910","-100,996.78516",302.99036,-0.00003,0.00001,"93,173.20035"
2023-03-01 09:00:00,buy,0.00600,-0.00002,"58,905.67029",0.00600,0.00600,-0.00000,0.00606,"3,051,425,182.50861","18,310,969.68910","55,874,554,025,485,456.00000",339.10294,"18,548,802.54985","-112,325.23990",0.50000,"36,622,957.10082","36,621,939.37821","-1,017.72261",-678.61966,-0.00002,-0.00002,0.98781,0.00000,"-58,905.67029",354.53750,1.06361,-0.00003,0.00001,"93,173.20035"
2023-03-01 10:00:00,buy,0.00602,-0.00390,"11,844,184.55520",0.00605,0.00605,0.00000,0.00608,"3,039,580,997.95341","18,382,321.13673","55,874,554,025,485,456.00000",553.80138,"6,704,617.99466","-40,759.09383",0.50000,"36,764,854.18623","36,764,642.27346",-211.91277,341.88861,0.00001,-0.00000,0.99562,279.70234,"-11,844,184.55520","71,566.14606",214.69844,-0.00001,0.00002,"93,452.90269"
2023-03-01 10:00:00,buy,0.00605,-0.00001,"23,033.74828",0.00605,0.00605,0.00000,0.00608,"3,039,557,964.20513","18,382,460.43783","55,874,554,025,485,456.00000",554.22054,"6,681,584.24637","-40,619.37358",0.50000,"36,765,131.75579","36,764,920.87566",-210.88013,343.34041,0.00001,-0.00000,0.99562,0.00000,"-23,033.74828",139.72026,0.41916,-0.00001,0.00002,"93,452.90269"
2023-03-01 14:00:00,buy,0.00607,-0.00334,"10,124,666.16022",0.00609,0.00609,0.00000,0.00610,"3,029,433,298.04490","18,443,896.44147","55,874,554,025,485,456.00000",739.08314,"-3,443,081.91385","21,001.49267",0.50000,"36,887,753.67596","36,887,792.88294",39.20698,778.29012,0.00002,-0.00000,1.00229,206.55901,"-10,124,666.16022","61,620.86624",184.86260,0.00000,0.00002,"93,659.46170"
